In [4]:
import sqlite3
import pandas as pd

In [ ]:
conn = sqlite3.connect('database.sqlite')


In [22]:
data = pd.read_sql_query("""
SELECT match_api_id, league_id,
        season, stage, date,
        home_team_api_id,
        away_team_api_id,
        home_team_goal,
        away_team_goal
    FROM Match
    WHERE season == '2008/2009' AND league_id == 1
    ORDER BY stage
""", conn)

In [23]:
print(data)

     match_api_id  league_id     season  stage                 date  \
0          492473          1  2008/2009      1  2008-08-17 00:00:00   
1          492474          1  2008/2009      1  2008-08-16 00:00:00   
2          492475          1  2008/2009      1  2008-08-16 00:00:00   
3          492476          1  2008/2009      1  2008-08-17 00:00:00   
4          492477          1  2008/2009      1  2008-08-16 00:00:00   
5          492478          1  2008/2009      1  2008-09-24 00:00:00   
6          492479          1  2008/2009      1  2008-08-16 00:00:00   
7          492480          1  2008/2009      1  2008-08-16 00:00:00   
8          492481          1  2008/2009      1  2008-08-16 00:00:00   
9          492482          1  2008/2009      2  2008-10-29 00:00:00   
10         492483          1  2008/2009      2  2008-08-23 00:00:00   
11         492484          1  2008/2009      2  2008-08-23 00:00:00   
12         492485          1  2008/2009      2  2008-08-24 00:00:00   
13    

In [47]:
team = {}
for row in conn.execute("""
    SELECT team
    FROM (
        SELECT home_team_api_id as team
        FROM Match
        WHERE season == '2008/2009' AND league_id == 1
        UNION
        SELECT away_team_api_id as team
        FROM Match
        WHERE season == '2008/2009' AND league_id == 1
    ) team
"""):
    team[row[0]] = 0    

In [49]:
print(team)

{4049: 0, 7947: 0, 8203: 0, 8342: 0, 8571: 0, 8635: 0, 9984: 0, 9985: 0, 9986: 0, 9987: 0, 9991: 0, 9993: 0, 9994: 0, 9996: 0, 9998: 0, 9999: 0, 10000: 0, 10001: 0}


In [146]:
train = data[data['stage'] < 30]
predict = data[data['stage'] >= 30]


In [147]:
def calculate_G(home, away):
    dif = abs(home - away)
    if dif == 0 or dif == 1:
        return 1
    elif dif == 2:
        return 1.5
    else:
        return (11 + dif) / 8


def calculate_W(first, second):
    dif = team[first] - team[second]
    return 1 / (10 ** (-dif / float(sum(team.values())) / len(team)) + 1)

def train_rating(row):
    G = calculate_G(row['home_team_goal'], row['away_team_goal'])
    if row['home_team_goal'] == row['away_team_goal']:
        team[row['home_team_api_id']] += G * (0.5 - calculate_W(row['home_team_api_id'], row['away_team_api_id']))
        team[row['away_team_api_id']] += G * (0.5 - calculate_W(row['away_team_api_id'], row['home_team_api_id']))  
    elif row['home_team_goal'] > row['away_team_goal']:
        team[row['home_team_api_id']] += G * (1 - calculate_W(row['home_team_api_id'], row['away_team_api_id']))
        team[row['away_team_api_id']] += G * (0 - calculate_W(row['away_team_api_id'], row['home_team_api_id']))
    else:
        team[row['away_team_api_id']] += G * (1 - calculate_W(row['away_team_api_id'], row['home_team_api_id']))
        team[row['home_team_api_id']] += G * (0 - calculate_W(row['home_team_api_id'], row['away_team_api_id']))

In [148]:
for _, row in train.iterrows():
   train_rating(row)

In [149]:
print(team)

{4049: -96.18352793802246, 7947: -10.153291007757094, 8203: 30.118115536218042, 8342: 68.589384452268, 8571: -30.739432712122053, 8635: 198.97976399205754, 9984: 8.86333990295979, 9985: 188.94403960774173, 9986: 26.450956130965366, 9987: 24.342906826304528, 9991: 120.7919300900258, 9993: 29.9017485446415, 9994: 62.52084088221111, 9996: 3.794302609388759, 9998: -98.65398464553334, 9999: -44.10570711496464, 10000: 99.79866771047617, 10001: 46.87315207150086}


In [150]:
accuracy = [0, 0]
for _, row in predict.iterrows():
    delta = 0.1 * float(sum(team.values())) / len(team)
    
    if abs(team[row['home_team_api_id']] - team[row['away_team_api_id']]) < delta:
        if row['home_team_goal'] == row['away_team_goal']:
            accuracy[0] += 1
        else:
            accuracy[1] += 1
    elif row['home_team_goal'] == row['away_team_goal']:
            accuracy[1] += 1        
    elif row['home_team_goal'] > row['away_team_goal'] and team[row['home_team_api_id']] > team[row['away_team_api_id']]:
        accuracy[0] += 1
    elif row['home_team_goal'] < row['away_team_goal'] and team[row['home_team_api_id']] < team[row['away_team_api_id']]:
        accuracy[0] += 1
    else:
        accuracy[1] += 1
    
    train_rating(row)

In [151]:
print(accuracy)
print(accuracy[0] / (accuracy[0] + accuracy[1]))

[24, 21]
0.5333333333333333


In [96]:
print(team)

{4049: 7.811273927991962, 7947: 17.916669949444618, 8203: 23.595843101927073, 8342: 52.50419199330039, 8571: 22.15430195443111, 8635: 80.6161245881535, 9984: 33.44694996628317, 9985: 70.65814188137522, 9986: 24.149429977215057, 9987: 42.970663870977326, 9991: 50.958887980931394, 9993: 31.512352186516686, 9994: 39.7386130634657, 9996: 32.3740942883896, 9998: 8.048955648278035, 9999: 8.622391867443971, 10000: 47.97023252599953, 10001: 34.99770368998034}
